# Import

In [29]:
import numpy as np
import random
import os
import math

from glob import glob
import pandas as pd
import cv2
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import torchvision.models as models
from torchvision import transforms

In [30]:
device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameters Setting

In [31]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':1000,
    'LEARNING_RATE':3e-2,
    'BATCH_SIZE':8,
    'SEED':41,
    'MODEL':'regnet',
    'NUM_CLASSES':1,
    'FEATURE_EXTRACT':True,
    'PATIENCE': 100
}


# Fix RandomSeed

In [32]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

# Data Pre-processing

In [33]:
def get_train_data(data_dir):
    img_path_list = []
    label_list = []
    list_dir = os.listdir(data_dir)
    for case_name in list_dir:
        current_path = os.path.join(data_dir, case_name)
        if os.path.isdir(current_path):
            # get image path
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.jpg')))
            img_path_list.extend(glob(os.path.join(current_path, 'image', '*.png')))
            del img_path_list[-1]
            # get label
            label_df = pd.read_csv(current_path+'\label.csv')
            label_list.extend(label_df['leaf_weight'])
            del label_list[0]
                
    return img_path_list, label_list

def get_test_data(data_dir):
    # get image path
    img_path_list = glob(os.path.join(data_dir, 'image', '*.png'))
    img_path_list.extend(glob(os.path.join(data_dir, 'image', '*.jpg')))
    img_path_list.sort(key=lambda x:int(x.split('\\')[-1].split('.')[0]))
    return img_path_list

In [34]:
all_img_path, all_label = get_train_data('C:\Dev\Python\pytorch\dacon(Optimize Growth Environment Competition)\data\\train')
test_img_path = get_test_data('C:\Dev\Python\pytorch\dacon(Optimize Growth Environment Competition)\data\\test')

# Train / Validation Split

In [35]:
# Train : Validation = 0.8 : 0.2 Split
train_len = int(len(all_img_path)*0.8)

train_img_path = all_img_path[:train_len]
train_label = all_label[:train_len]

vali_img_path = all_img_path[train_len:]
vali_label = all_label[train_len:]

# CustomDataset

In [36]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, train_mode=True, transforms=None):
        self.transforms = transforms
        self.train_mode = train_mode
        self.img_path_list = img_path_list
        self.label_list = label_list

    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        # Get image data
        image = cv2.imread(img_path)
        if self.transforms is not None:
            image = self.transforms(image)

        if self.train_mode:
            label = self.label_list[index]
            return image, label
        else:
            return image
    
    def __len__(self):
        return len(self.img_path_list)

In [37]:
train_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                    ])

test_transform = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                    ])

In [38]:
# Get Dataloader
train_dataset = CustomDataset(train_img_path, train_label, train_mode=True, transforms=train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

vali_dataset = CustomDataset(vali_img_path, vali_label, train_mode=True, transforms=test_transform)
vali_loader = DataLoader(vali_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Define Model Architecture

In [39]:
import torchvision.models as models
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None

    if model_name == 'regnet':
        model_ft = models.regnet_y_32gf(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)

    return model_ft

# class CNNRegressor(torch.nn.Module):
#     def __init__(self):
#         super(CNNRegressor, self).__init__()
#         self.layer1 = torch.nn.Sequential(
#             nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2))
        
#         self.layer2 = torch.nn.Sequential(
#             nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2))
        
#         self.layer3 = torch.nn.Sequential(
#             nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2))
        
#         self.layer4 = torch.nn.Sequential(
#             nn.Conv2d(32, 64, kernel_size=4, stride=1, padding=1),
#             nn.ReLU(),
#             nn.MaxPool2d(kernel_size=2, stride=2))
        
#         self.regressor = nn.Linear(int((CFG['IMG_SIZE']/16-1)**2*64),1)


#     def forward(self, x):
#         # Simple CNN Model (Batch, 3, 128, 128 -> Batch, 64, 7, 7)
#         # (Batch, 3, 128, 128)
#         x = self.layer1(x)
#         # (Batch, 8, 64, 64)
#         x = self.layer2(x)
#         # (Batch, 16, 32, 32)
#         x = self.layer3(x)
#         # (Batch, 32, 16, 16)
#         x = self.layer4(x)
#         # (Batch, 64, 7, 7) -> Flatten (Batch, 64*7*7(=3136))
#         x = torch.flatten(x, start_dim=1)
#         # Regressor (Batch, 3136) -> (Batch, 1)
#         out = self.regressor(x)
#         return out

# Early stopping class

In [40]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, epoch = 0, patience=20, verbose=False, delta=0, path='C:\Dev\Python\pytorch\dacon(Optimize Growth Environment Competition)\model\\tmp\\'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.epoch = epoch

    def __call__(self, epoch, val_loss, model):
        self.epoch = epoch
        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score > self.best_score + self.delta: # 이 코드에서는 MAE를 쓰는데 값이 작은게 좋은 거니까.
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), f'{self.path}best_model_{self.best_score}_{self.epoch}_AdamW.pth')
        self.val_loss_min = val_loss

# Train

In [41]:
def train(model, optimizer, train_loader, vali_loader, scheduler, device):
    model.to(device)

    # Loss Function
    criterion = nn.L1Loss().to(device)
    
    # early_stopping
    early_stopping = EarlyStopping(patience = CFG['PATIENCE'], verbose = True)

    for epoch in range(1,CFG["EPOCHS"]+1):
        model.train()
        train_loss = []
        for img, label in tqdm(iter(train_loader)):
            img, label = img.float().to(device), label.float().to(device)
            
            optimizer.zero_grad()

            # Data -> Model -> Output
            logit = model(img)
            # Calc loss
            loss = criterion(logit.squeeze(1), label)

            # backpropagation
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())
            
        if scheduler is not None:
            scheduler.step()
            
        # Evaluation Validation set
        vali_mae = validation(model, vali_loader, criterion, device)
        
        print(f'Epoch [{epoch}] Train MAE : [{np.mean(train_loss):.5f}] Validation MAE : [{vali_mae:.5f}]\n')

        # early_stopping는 validation loss가 감소하였는지 확인이 필요하며,
        # 만약 감소하였을경우 현제 모델을 checkpoint로 만든다.
        early_stopping(epoch, vali_mae, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

In [42]:
def validation(model, vali_loader, criterion, device):
    model.eval() # Evaluation
    vali_loss = []
    with torch.no_grad():
        for img, label in tqdm(iter(vali_loader)):
            img, label = img.float().to(device), label.float().to(device)

            logit = model(img)
            loss = criterion(logit.squeeze(1), label)
            
            vali_loss.append(loss.item())

    vali_mae_loss = np.mean(vali_loss)
    return vali_mae_loss

# Run

In [43]:
import torch_optimizer as optim

model_ft = initialize_model(CFG['MODEL'], CFG['NUM_CLASSES'], CFG['FEATURE_EXTRACT'], use_pretrained=True)
model_ft.load_state_dict(torch.load('C:\Dev\Python\pytorch\dacon(Optimize Growth Environment Competition)\model\\tmp\\best_model_74.17234217493157_2_AdamW.pth'))
model = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if CFG['FEATURE_EXTRACT']:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)


# optimizer = optim.Lamb(
#     params = model.parameters(),
#     lr = CFG["LEARNING_RATE"],
#     betas=(0.9, 0.999),
#     eps=1e-8,
#     weight_decay=0,
    
# )
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"],)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.002, 
                                                steps_per_epoch=10, epochs=CFG['EPOCHS'],anneal_strategy='cos')

train(model, optimizer, train_loader, vali_loader, scheduler, device)

Params to learn:
	 fc.weight
	 fc.bias


  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

Epoch [1] Train MAE : [67.22158] Validation MAE : [74.32943]

Validation loss decreased (inf --> 74.329431).  Saving model ...


  0%|          | 0/152 [00:00<?, ?it/s]

...

  0%|          | 0/38 [00:00<?, ?it/s]

Epoch [999] Train MAE : [59.51197] Validation MAE : [71.63003]

EarlyStopping counter: 50 out of 100


  0%|          | 0/152 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

Epoch [1000] Train MAE : [59.15418] Validation MAE : [72.27826]

EarlyStopping counter: 51 out of 100


# Inference

In [14]:
def predict(model, test_loader, device):
    model.eval()
    model_pred = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.float().to(device)

            pred_logit = model(img)
            pred_logit = pred_logit.squeeze(1).detach().cpu()

            model_pred.extend(pred_logit.tolist())
    return model_pred

In [15]:
test_dataset = CustomDataset(test_img_path, None, train_mode=False, transforms=test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Validation Score가 가장 뛰어난 모델을 불러옵니다.
checkpoint = torch.load('./saved/best_model.pth')
model = CNNRegressor().to(device)
model.load_state_dict(checkpoint)

# Inference
preds = predict(model, test_loader, device)

NameError: name 'CustomDataset' is not defined

# Submission

In [16]:
submission = pd.read_csv('./sample_submission.csv')
submission['leaf_weight'] = preds
submission.to_csv('./submit.csv', index=False)

NameError: name 'pd' is not defined